In [ ]:
from v2realbot.tools.loadbatch import load_batch
from v2realbot.utils.utils import zoneNY
import pandas as pd
import numpy as np
import vectorbtpro as vbt

vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280
vbt.settings.plotting.auto_rangebreaks = True
# Set the option to display with pagination
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', 10)  # Number of rows per page

res, df = load_batch(batch_id="e44a5075",
                     space_resolution_evenly=False,
                     indicators_columns=["Rsi14"],
                     main_session_only=True)
if res < 0:
    print("Error" + str(res) + str(df))
df = df["bars"]

#df

# filter dates

In [ ]:
#naloadujeme do vbt symbol as column
basic_data = vbt.Data.from_data({"BAC": df}, tz_convert=zoneNY)
start_date = pd.Timestamp('2024-03-12 09:30', tz=zoneNY)
end_date = pd.Timestamp('2024-03-13 16:00', tz=zoneNY)

#basic_data = basic_data.transform(lambda df: df[df.index.date == start_date.date()])
basic_data = basic_data.transform(lambda df: df[(df.index >= start_date) & (df.index <= end_date)])
#basic_data.data["BAC"].info()

# fig = basic_data.plot(plot_volume=False)
# pivot_info = basic_data.run("pivotinfo", up_th=0.003, down_th=0.002)
# #pivot_info.plot()
# pivot_info.plot(fig=fig, conf_value_trace_kwargs=dict(visible=True))
# fig.show()


# rsi14 = basic_data.data["BAC"]["Rsi14"].rename("Rsi14")

# rsi14.vbt.plot().show()
# basic_data.data["BAC"].vbt.ohlcv.plot().show()

In [ ]:
@vbt.njit
def long_entry_place_func_nb(c, low, close, time_in_ns, rsi14, window_open, window_close):
    market_open_minutes = 570  # 9 hours * 60 minutes + 30 minutes

    for out_i in range(len(c.out)):
        i = c.from_i + out_i

        current_minutes = vbt.dt_nb.hour_nb(time_in_ns[i]) * 60 + vbt.dt_nb.minute_nb(time_in_ns[i])
        #print("current_minutes", current_minutes)
        # Calculate elapsed minutes since market open at 9:30 AM
        elapsed_from_open = current_minutes - market_open_minutes
        elapsed_from_open = elapsed_from_open if elapsed_from_open >= 0 else 0
        #print( "elapsed_from_open", elapsed_from_open)

        #elapsed_from_open = elapsed_minutes_from_open_nb(time_in_ns) 
        in_window = elapsed_from_open > window_open and elapsed_from_open < window_close
        #print("in_window", in_window)
        # if in_window:
        #     print("in window")

        if in_window and rsi14[i] > 60: # and low[i, c.col] <= hit_price: # and hour == 9:  # (4)!
            return out_i
    return -1

@vbt.njit
def long_exit_place_func_nb(c, high, close, time_index, tp, sl):  # (5)!
    entry_i = c.from_i - c.wait
    entry_price = close[entry_i, c.col]
    hit_price = entry_price * (1 + tp)
    stop_price = entry_price * (1 - sl)
    for out_i in range(len(c.out)):
        i = c.from_i + out_i
        last_bar_of_day = vbt.dt_nb.day_changed_nb(time_index[i], time_index[i + 1])

        #print(next_day)
        if last_bar_of_day: #pokud je dalsi next day, tak zavirame posledni
            print("ted",out_i)
            return out_i
        if close[i, c.col] >= hit_price or close[i, c.col] <= stop_price :
            return out_i
    return -1




In [ ]:
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))

df

In [ ]:
df.sum()